# YOLO 細節理解 - 網路輸出的後處理
今天的課程，我們說明了NMS在yolo中運作的情形:

NMS在YOLO的實際運作以每一個類別為主，各別執行NMS。
YOLO 在NMS中採用的信心度為「每個 bbox 包含各類別的信心度」

## 作業
在NMS流程中，IoU重疊率參數(nms_threshold )調高，試著思考一下輸出的預測框會有甚麼變化? Hint: 可以回頭看今天介紹的內容，思考輸出的預測框變多或變少?

In [1]:
import cv2
import numpy as np


def nms(bounding_boxes, confidence_score, threshold):
    if len(bounding_boxes) == 0:
        return [], []
    
    boxes = np.array(bounding_boxes)
    
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]
    
    score = np.array(confidence_score)
    
    
    picked_boxes = []
    picked_score = []
    
    
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)
    order = np.argsort(score)
    
    while order.size > 0:
        index = order[-1]
        picked_boxes.append(bounding_boxes[index])
        picked_score.append(confidence_score[index])
        
        x1 = np.maximum(start_x[index], start_x[order[:-1]])
        x2 = np.maximum(end_x[index], end_x[order[:-1]])
        y1 = np.maximum(start_y[index], start_y[order[:-1]])
        y2 = np.maximum(end_y[index], end_y[order[:-1]])
        
        w = np.maximum(0.0, x2 - x1 + 1)
        h = np.maximum(0.0, y2 - y1 + 1)
        intersection = w * h
        
        ratio = intersection / (areas[index] + areas[order[:-1]] - intersection)
        left = np.where(ratio < threshold)
        
        order = order[left]
        
    return picked_boxes, picked_score



image_name = '../data/nms.jpg'


bounding_boxes = [(187, 82, 337, 317), (150, 67, 305, 282), (246, 121, 368, 304)]
confidence_score = [0.9, 0.75, 0.8]

image = cv2.imread(image_name)

org = image.copy()

# Draw parameters
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 2

# IoU threshold
threshold = 0.9

# Draw bounding boxes and confidence score
for (start_x, start_y, end_x, end_y), confidence in zip(bounding_boxes, confidence_score):
    (w, h), baseline = cv2.getTextSize(str(confidence), font, font_scale, thickness)
    cv2.rectangle(org, (start_x, start_y - (2 * baseline + 5)), (start_x + w, start_y), (0, 255, 255), -1)
    cv2.rectangle(org, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)
    cv2.putText(org, str(confidence), (start_x, start_y), font, font_scale, (0, 0, 0), thickness)

# Run non-max suppression algorithm
picked_boxes, picked_score = nms(bounding_boxes, confidence_score, threshold)

# Draw bounding boxes and confidence score after non-maximum supression
for (start_x, start_y, end_x, end_y), confidence in zip(picked_boxes, picked_score):
    (w, h), baseline = cv2.getTextSize(str(confidence), font, font_scale, thickness)
    cv2.rectangle(image, (start_x, start_y - (2 * baseline + 5)), (start_x + w, start_y), (0, 255, 255), -1)
    cv2.rectangle(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)
    cv2.putText(image, str(confidence), (start_x, start_y), font, font_scale, (0, 0, 0), thickness)

# Show image
cv2.imshow('Original', org)
cv2.imshow('NMS=0.4', image)
cv2.waitKey(0)

27

設定IOU threshold是為了篩選掉其他的bounding box，如果IOU threshold愈來愈大會造成最後的bounding box愈來愈多